In [7]:
import xarray as xr
import numpy as np

In [2]:
def get_stats(data, section, southdiv):
    #function to get the average transport at each section over a given season
    #south div is to set up boolean for the three south water masses, 1=CUC, 2=south shelf/davidson, 3=Columbia, 0=NA 

    if section == 0:
        boolean = (data.final_section == section) & ((data.init_t-data.final_t)>24) # to remove tidally pumped parcels across PRT
    elif section == 2:
        high = 33.5 # salinity division between CUC and south shelf
        low = 32 # salinity division between south shelf and Columbia
        if southdiv == 1: # CUC
            boolean = (data.final_section == section) & (data.final_salt >= high)
        elif southdiv == 2:
            boolean = (data.final_section == section) & (data.final_salt < high) & (data.final_salt > low)
        elif southdiv == 3:
            boolean = (data.final_section == section) & (data.final_salt <= low)
        else:
            print('the options for the southern boundary are 1=CUC, 2=south shelf/davidson, 3=Columbia')
    else:
        boolean = (data.final_section == section)

    tran = np.sum(data.init_transp[boolean])/(np.max(data.init_t)-1440)

    return float(tran.values)


In [3]:
orig = xr.open_dataset('/data1/bbeutel/LO_user/ariane/OSM/20171020/ariane_positions_quantitative.nc')
orig

<xarray.Dataset> Size: 501MB
Dimensions:        (ntraj: 2846258)
Dimensions without coordinates: ntraj
Data variables: (12/25)
    init_x         (ntraj) float64 23MB ...
    init_y         (ntraj) float64 23MB ...
    init_z         (ntraj) float64 23MB ...
    init_t         (ntraj) float64 23MB ...
    init_age       (ntraj) timedelta64[ns] 23MB ...
    init_transp    (ntraj) float64 23MB ...
    ...             ...
    init_lon       (ntraj) float64 23MB ...
    init_lat       (ntraj) float64 23MB ...
    init_depth     (ntraj) float64 23MB ...
    final_lon      (ntraj) float64 23MB ...
    final_lat      (ntraj) float64 23MB ...
    final_depth    (ntraj) float64 23MB ...
Attributes: (12/112)
    key_roms:                  .TRUE.
    roms_global_attribute:     .TRUE.
    key_mars:                  .FALSE.
    key_symphonie:             .FALSE.
    key_B2C_grid:              .FALSE.
    key_sequential:            .TRUE.
    ...                        ...
    ind0_sa:                   1
    indn_sa:                   5616
    maxsize_sa:                5
    c_suffix_sa:               .nc
    nc_var_sa:                 salt
    nc_att_mask_sa:            NONE

In [6]:
comp = xr.open_dataset('/data1/bbeutel/LO_user/ariane/OSM/20171020/ariane_positions_quantitative_comp.nc')
comp

<xarray.Dataset> Size: 501MB
Dimensions:        (ntraj: 2846258)
Dimensions without coordinates: ntraj
Data variables: (12/25)
    final_age      (ntraj) timedelta64[ns] 23MB ...
    final_dens     (ntraj) float32 11MB ...
    final_depth    (ntraj) float64 23MB ...
    final_lat      (ntraj) float64 23MB ...
    final_lon      (ntraj) float64 23MB ...
    final_salt     (ntraj) float32 11MB ...
    ...             ...
    init_t         (ntraj) float64 23MB ...
    init_temp      (ntraj) float32 11MB ...
    init_transp    (ntraj) float64 23MB ...
    init_x         (ntraj) float64 23MB ...
    init_y         (ntraj) float64 23MB ...
    init_z         (ntraj) float64 23MB ...
Attributes: (12/114)
    key_roms:                  .TRUE.
    roms_global_attribute:     .TRUE.
    key_mars:                  .FALSE.
    key_symphonie:             .FALSE.
    key_B2C_grid:              .FALSE.
    key_sequential:            .TRUE.
    ...                        ...
    maxsize_sa:                5
    c_suffix_sa:               .nc
    nc_var_sa:                 salt
    nc_att_mask_sa:            NONE
    history:                   Thu Jun 13 12:55:32 2024: ncks -4 -L4 ariane_p...
    NCO:                       netCDF Operators version 4.7.5 (Homepage = htt...

In [8]:
# accessing the data (transport)
section=['North','Offshore','CUC','South','Columbia','loop']
otran = [get_stats(orig,4,0),get_stats(orig,3,0),get_stats(orig,2,1),get_stats(orig,2,2),get_stats(orig,2,3),get_stats(orig,0,0)]

In [9]:
ctran = [get_stats(comp,4,0),get_stats(comp,3,0),get_stats(comp,2,1),get_stats(comp,2,2),get_stats(comp,2,3),get_stats(comp,0,0)]

In [11]:
np.array(otran) - np.array(ctran)

array([0., 0., 0., 0., 0., 0.])

files look the same to me, compressed one took more than twice as long (23s versus 10s) to extra transport info